In [65]:
from Bio import Entrez, GenBank, SeqIO
Entrez.email = "gsyu93@gmail.com"

In [28]:
handle = Entrez.esearch(db="clinvar", term='VCV000209223', idtype='acc')
result = Entrez.read(handle)

handle.close()
# records = Entrez.parse(handle)

In [29]:
result

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['209223'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'VCV000209223[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'VCV000209223[All Fields]'}

In [4]:
handle = Entrez.efetch(db='clinvar', id='209223', rettype='vcv', is_varationid='true', from_esearch="true")
handle.read()


b'<?xml version="1.0" encoding="UTF-8" ?>\n                        \n<ClinVarResult-Set><VariationArchive VariationID="209223" VariationName="NM_007294.3(BRCA1):c.*4549G&gt;C" VariationType="single nucleotide variant" Accession="VCV000209223" Version="3" RecordType="interpreted" NumberOfSubmissions="1" NumberOfSubmitters="1" DateLastUpdated="2022-11-05" DateCreated="2015-09-29" MostRecentSubmission="2015-09-29">\n  <RecordStatus>current</RecordStatus>\n  <Species>Homo sapiens</Species>\n  <InterpretedRecord>\n    <SimpleAllele AlleleID="206181" VariationID="209223">\n      <GeneList>\n        <Gene Symbol="BRCA1" FullName="BRCA1 DNA repair associated" GeneID="672" HGNC_ID="HGNC:1100" Source="submitted" RelationshipType="near gene, downstream">\n          <Location>\n            <CytogeneticLocation>17q21.31</CytogeneticLocation>\n            <SequenceLocation Assembly="GRCh38" AssemblyAccessionVersion="GCF_000001405.38" AssemblyStatus="current" Chr="17" Accession="NC_000017.11" start="

In [5]:
handle = Entrez.efetch(db='clinvar', id='VCV000209223', rettype='vcv')

In [6]:

import xml.etree.ElementTree as ET
result = ET.parse(handle)
root = result.getroot()

In [7]:
def _pretty_print(current, parent=None, index=-1, depth=0):
    for i, node in enumerate(current):
        _pretty_print(node, current, i, depth + 1)
    if parent is not None:
        if index == 0:
            parent.text = '\n' + ('\t' * depth)
        else:
            parent[index - 1].tail = '\n' + ('\t' * depth)
        if index == len(parent) - 1:
            current.tail = '\n' + ('\t' * (depth - 1))

_pretty_print(root)

ET.dump(result)

<ClinVarResult-Set>
	<VariationArchive VariationID="209223" VariationName="NM_007294.3(BRCA1):c.*4549G&gt;C" VariationType="single nucleotide variant" Accession="VCV000209223" Version="3" RecordType="interpreted" NumberOfSubmissions="1" NumberOfSubmitters="1" DateLastUpdated="2022-11-05" DateCreated="2015-09-29" MostRecentSubmission="2015-09-29">
		<RecordStatus>current</RecordStatus>
		<Species>Homo sapiens</Species>
		<InterpretedRecord>
			<SimpleAllele AlleleID="206181" VariationID="209223">
				<GeneList>
					<Gene Symbol="BRCA1" FullName="BRCA1 DNA repair associated" GeneID="672" HGNC_ID="HGNC:1100" Source="submitted" RelationshipType="near gene, downstream">
						<Location>
							<CytogeneticLocation>17q21.31</CytogeneticLocation>
							<SequenceLocation Assembly="GRCh38" AssemblyAccessionVersion="GCF_000001405.38" AssemblyStatus="current" Chr="17" Accession="NC_000017.11" start="43044295" stop="43125364" display_start="43044295" display_stop="43125364" Strand="-" />
							

In [8]:
root = result.getroot()
root.attrib

{}

In [84]:
var_loc = root.findall('./VariationArchive/InterpretedRecord/SimpleAllele/Location/SequenceLocation')

for info in var_loc:
    print(info)
    print(info.tag)
    print(info.attrib)
    print(info.attrib['Assembly'])
    if info.attrib['Assembly'] == 'GRCh38':
        chr_acc = info.attrib['Accession']
        start   = int(info.attrib['start'])
        ref_nt  = info.attrib['referenceAlleleVCF']
        alt_nt  = info.attrib['alternateAlleleVCF']

        break


<Element 'SequenceLocation' at 0x0000025029C244F0>
SequenceLocation
{'Assembly': 'GRCh38', 'AssemblyAccessionVersion': 'GCF_000001405.38', 'forDisplay': 'true', 'AssemblyStatus': 'current', 'Chr': '17', 'Accession': 'NC_000017.11', 'start': '43041129', 'stop': '43041129', 'display_start': '43041129', 'display_stop': '43041129', 'variantLength': '1', 'positionVCF': '43041129', 'referenceAlleleVCF': 'C', 'alternateAlleleVCF': 'G'}
GRCh38


In [85]:
chr_seq_fetch = Entrez.efetch(db="nucleotide", 
                              id="NC_000017", 
                              rettype="fasta", 
                              strand=1, 
                              seq_start=start-60, 
                              seq_stop=start+60)

record = SeqIO.read(chr_seq_fetch, "fasta")
chr_seq_fetch.close()
print(record.seq)

TCTGCTCCTAGATTGCTAATATAATTTAATCTTTGTCCTTTGCATTGTACAGCAGTCCCCCCTTATCTGCGGGGCATATGTTCCAAGACCACAAGTAGATGCCTGAACCCTCGGATAGTAC
